## 1. Import Libraries

In [5]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import sys
sys.path.append('/work3/s233791/rl-pricing-amod')
from src.envs.amod_env_multi import Scenario

# Set plot style for professional appearance
plt.style.use('seaborn-v0_8-paper')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Define City Parameters

These parameters match those used in the choice model calibration notebook.

In [6]:
# Define city-specific parameters (from choice_model_calibration notebook)
cities = ['san_francisco', 'washington_dc', 'nyc_man_south']
city_names = {
    'nyc_man_south': 'NYC Manhattan South',
    'washington_dc': 'Washington DC',
    'san_francisco': 'San Francisco'
}

# Scenario initialization parameters
demand_ratio = {'san_francisco': 2, 'nyc_man_south': 1.0, 'washington_dc': 4.2}
json_hr = {'san_francisco': 19, 'nyc_man_south': 19, 'washington_dc': 19}
json_tstep = {'san_francisco': 3, 'nyc_man_south': 3, 'washington_dc': 3}

# Common parameters
TF = 20
IMPUTE = 0
SUPPLY_RATIO = 1.0
FIX_PRICE = True

# Number of realizations for averaging
N_REALIZATIONS = 10

print("City-specific parameters:")
for city in cities:
    print(f"\n{city_names[city]}:")
    print(f"  Demand ratio: {demand_ratio[city]}")
    print(f"  JSON hour: {json_hr[city]}")
    print(f"  JSON timestep: {json_tstep[city]} minutes")

print(f"\nNumber of realizations: {N_REALIZATIONS}")

City-specific parameters:

San Francisco:
  Demand ratio: 2
  JSON hour: 19
  JSON timestep: 3 minutes

Washington DC:
  Demand ratio: 4.2
  JSON hour: 19
  JSON timestep: 3 minutes

NYC Manhattan South:
  Demand ratio: 1.0
  JSON hour: 19
  JSON timestep: 3 minutes

Number of realizations: 10


## 3. Initialize Scenarios for All Cities

In [ ]:
# Create scenario objects for all cities
scenarios = {}

for city in cities:
    print(f"Initializing scenario for {city_names[city]}...")
    scenarios[city] = Scenario(
        json_file=f"data/scenario_{city}.json",
        json_hr=json_hr[city],
        json_tstep=json_tstep[city],
        tf=TF,
        impute=IMPUTE,
        demand_ratio=demand_ratio[city]*2,
        supply_ratio=SUPPLY_RATIO,
        fix_price=FIX_PRICE
    )
print("\nAll scenarios initialized successfully!")

Initializing scenario for San Francisco...
Initializing scenario for Washington DC...
Initializing scenario for NYC Manhattan South...

All scenarios initialized successfully!


## 4. Analyze Raw Demand and Variation

For each city, we run 10 realizations and calculate:
- **Total raw demand**: Sum of all passengers across all origin-destination pairs
- **Regional demand**: Demand aggregated by origin region
- **Coefficient of variation (CV)**: Standard deviation / mean of regional demand

In [10]:
# Store results for all cities
results = {}

for city in cities:
    print(f"\n{'='*80}")
    print(f"Analyzing {city_names[city]}")
    print(f"{'='*80}")
    
    scenario = scenarios[city]
    n_regions = scenario.nregion
    
    # Store metrics for each realization
    total_demands = []
    regional_demands_list = []
    cvs = []
    
    # Run multiple realizations
    for realization in tqdm(range(N_REALIZATIONS), desc=f"Running realizations"):
        # Get random demand sample (Poisson sampling)
        trip_data = scenario.get_random_demand(reset=False)
        
        # Calculate total demand and regional demand
        total_demand = 0
        regional_demand = np.zeros(n_regions)
        
        # trip_data is a list of tuples: (origin, destination, time_idx, demand, price)
        for origin, destination, time_idx, demand, price in trip_data:
            total_demand += demand
            regional_demand[origin] += demand
        
        # Calculate coefficient of variation (std / mean)
        mean_regional_demand = np.mean(regional_demand)
        std_regional_demand = np.std(regional_demand)
        cv = std_regional_demand / mean_regional_demand if mean_regional_demand > 0 else 0
        
        # Store results
        total_demands.append(total_demand)
        regional_demands_list.append(regional_demand)
        cvs.append(cv)
    
    # Calculate averages across realizations
    avg_total_demand = np.mean(total_demands)
    std_total_demand = np.std(total_demands)
    avg_cv = np.mean(cvs)
    std_cv = np.std(cvs)
    avg_regional_demand = np.mean(regional_demands_list, axis=0)
    
    # Store results
    results[city] = {
        'total_demands': total_demands,
        'avg_total_demand': avg_total_demand,
        'std_total_demand': std_total_demand,
        'cvs': cvs,
        'avg_cv': avg_cv,
        'std_cv': std_cv,
        'avg_regional_demand': avg_regional_demand,
        'n_regions': n_regions
    }
    
    print(f"\nResults for {city_names[city]}:")
    print(f"  Average total demand: {avg_total_demand:.1f} ± {std_total_demand:.1f} passengers")
    print(f"  Average CV: {avg_cv:.3f} ± {std_cv:.3f}")
    print(f"  Number of regions: {n_regions}")
    print(f"  Average demand per region: {avg_total_demand/n_regions:.1f} passengers")

print(f"\n{'='*80}")
print("Analysis complete for all cities!")
print(f"{'='*80}")


Analyzing San Francisco


Running realizations: 100%|██████████| 10/10 [00:00<00:00, 86.38it/s]



Results for San Francisco:
  Average total demand: 5490.1 ± 63.6 passengers
  Average CV: 1.305 ± 0.013
  Number of regions: 10
  Average demand per region: 549.0 passengers

Analyzing Washington DC


Running realizations: 100%|██████████| 10/10 [00:00<00:00, 26.35it/s]



Results for Washington DC:
  Average total demand: 16881.3 ± 135.7 passengers
  Average CV: 1.264 ± 0.008
  Number of regions: 18
  Average demand per region: 937.8 passengers

Analyzing NYC Manhattan South


Running realizations: 100%|██████████| 10/10 [00:00<00:00, 59.96it/s]


Results for NYC Manhattan South:
  Average total demand: 21269.9 ± 118.6 passengers
  Average CV: 0.694 ± 0.006
  Number of regions: 12
  Average demand per region: 1772.5 passengers

Analysis complete for all cities!


## 5. Summary Table

In [11]:
# Create summary dataframe
summary_data = []
for city in cities:
    summary_data.append({
        'City': city_names[city],
        'Number of Regions': results[city]['n_regions'],
        'Avg Total Demand': f"{results[city]['avg_total_demand']:.1f}",
        'Std Total Demand': f"{results[city]['std_total_demand']:.1f}",
        'Avg Demand per Region': f"{results[city]['avg_total_demand']/results[city]['n_regions']:.1f}",
        'Avg CV': f"{results[city]['avg_cv']:.3f}",
        'Std CV': f"{results[city]['std_cv']:.3f}"
    })

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*100)
print("SUMMARY: DEMAND CHARACTERISTICS ACROSS CITIES")
print("="*100)
print(f"\nAveraged over {N_REALIZATIONS} realizations")
print("\nCV = Coefficient of Variation (Std Dev / Mean of regional demand)")
print("Higher CV indicates more uneven demand distribution across regions\n")
display(summary_df)

# Export to CSV
summary_df.to_csv('demand_analysis_summary.csv', index=False)
print("\nSummary table saved to: demand_analysis_summary.csv")


SUMMARY: DEMAND CHARACTERISTICS ACROSS CITIES

Averaged over 10 realizations

CV = Coefficient of Variation (Std Dev / Mean of regional demand)
Higher CV indicates more uneven demand distribution across regions



,City,Number of Regions,Avg Total Demand,Std Total Demand,Avg Demand per Region,Avg CV,Std CV
0,San Francisco,10,5490.1,63.6,549.0,1.305,0.013
1,Washington DC,18,16881.3,135.7,937.8,1.264,0.008
2,NYC Manhattan South,12,21269.9,118.6,1772.5,0.694,0.006



Summary table saved to: demand_analysis_summary.csv
